In [1]:
import os
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Set working directory
os.chdir(os.path.expanduser("/Users/ericarmstrong/Desktop"))

# Load and parse dates
df = pd.read_csv("table_final.csv")
df['date'] = pd.to_datetime(df['date'])

# Group and average duplicate entries
df_clean = df.groupby(['date', 'seriesName'], as_index=False)['data'].mean()

# Pivot long → wide
df_wide = df_clean.pivot(index='date', columns='seriesName', values='data')
print("Available columns after pivot:\n", df_wide.columns.tolist())

# Rename columns for easier formula access
df_wide = df_wide.rename(columns={
    'Personal consumption expenditures (PCE)': 'PCE',
    'Food and beverages purchased for off-premises consumption': 'FNBOPC'
})

# Keep only target variables
target_vars = ['UFMEN', 'UFEC', 'NRSI511N', 'CCIN', 'NRSV2N', 'CCOND', 'RECPROB', 'VSMINN', 'VSMIDN', 
               'PCE', 'Education', 'PXEA', 'PMEA', 'FNBOPC']

for var in target_vars:
    if var not in df_wide.columns:
        raise ValueError(f"Missing required column: {var}")

# Filter and clean model data
df_model = df_wide[target_vars].copy()
df_model = df_model.apply(pd.to_numeric, errors='coerce').dropna()
df_model = df_model.pct_change().dropna()

# VIF calculation (exclude dependent var 'UFMEN')
X = df_model.drop(columns='UFMEN')
X = sm.add_constant(X)  # Add intercept term

vif_df = pd.DataFrame()
vif_df["Variable"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("\n=== Variance Inflation Factors ===")
print(vif_df.sort_values("VIF", ascending=False))

# Regression
formula5 = 'UFMEN ~ UFEC + PCE + VSMINN + CCOND + RECPROB + PXEA + PMEA + NRSI511N + VSMIDN + Education + FNBOPC'
model5 = smf.ols(formula5, data=df_model).fit()
print("\n=== OLS Regression Results ===")
print(model5.summary())

/Users/ericarmstrong/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ericarmstrong/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Available columns after pivot:
 ['Addenda:', 'CCIN', 'CCOND', 'CPIUANN', 'Clothing and footwear', 'Clothing, footwear, and related services', 'Communication', 'Durable goods', 'ERSI5N', 'Education', 'Energy goods and services5', 'Final consumption expenditures of nonprofit institutions serving households (NPISHs)1', 'Financial services and insurance', 'Food and beverages purchased for off-premises consumption', 'Food services and accommodations', 'Furnishings and durable household equipment', 'Furnishings, household equipment, and routine household maintenance', 'GDPA', 'Gasoline and other energy goods', 'Goods', 'Gross output of nonprofit institutions2', 'HLNCVP', 'Health', 'Health care', 'Household consumption expenditures (for services)', 'Household consumption expenditures7', 'Housing and utilities', 'Housing, utilities, and fuels', 'Less: Receipts from sales of goods and services by nonprofit institutions3', 'Market-based PCE excluding food and energy6', 'Market-based PCE6', 'Moto